In [27]:
import pandas as pd

# Load the dataset
df = pd.read_csv('gmb.csv', sep=",", encoding="ISO-8859-1")

# Fill NaN values in 'Sentence #' column with the previous value (forward fill)
df['Sentence #'] = df['Sentence #'].ffill()

# Check the first few rows to ensure data is loaded and cleaned correctly
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [28]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Replace NaN values in the 'Word' column with an empty string or placeholder
df['Word'] = df['Word'].fillna('')

# Ensure that all values in 'Word' are strings
df['Word'] = df['Word'].astype(str)

# Tokenizing the words (Tokens)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Word'].values)
X = tokenizer.texts_to_sequences(df['Word'].values)
X = pad_sequences(X, padding='post')

# Encode POS and NE labels
pos_encoder = LabelEncoder()
df['POS'] = pos_encoder.fit_transform(df['POS'])

ne_encoder = LabelEncoder()
df['Tag'] = ne_encoder.fit_transform(df['Tag'])

# Split the data into training and testing (90% train, 10% test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, df[['POS', 'Tag']], test_size=0.1, random_state=42)


AttributeError: 'float' object has no attribute 'lower'

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, SimpleRNN, Dense, Dropout, Embedding

# Define the model
def create_model(model_type='LSTM'):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))
    
    if model_type == 'LSTM':
        model.add(LSTM(128, return_sequences=True))
    elif model_type == 'BiLSTM':
        model.add(Bidirectional(LSTM(128, return_sequences=True)))
    elif model_type == 'RNN':
        model.add(SimpleRNN(128, return_sequences=True))

    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(len(pos_encoder.classes_), activation='softmax'))  # POS classification
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create models
lstm_model = create_model('LSTM')
bilstm_model = create_model('BiLSTM')
rnn_model = create_model('RNN')



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')

# Training LSTM Model
lstm_model.fit(X_train, y_train['POS'], epochs=10, batch_size=64, validation_split=0.1, callbacks=[early_stop, checkpoint])

# Training BiLSTM Model
bilstm_model.fit(X_train, y_train['POS'], epochs=10, batch_size=64, validation_split=0.1, callbacks=[early_stop, checkpoint])

# Training RNN Model
rnn_model.fit(X_train, y_train['POS'], epochs=10, batch_size=64, validation_split=0.1, callbacks=[early_stop, checkpoint])



In [ ]:
from sklearn.metrics import classification_report

# Predictions
lstm_pred = lstm_model.predict(X_test)
bilstm_pred = bilstm_model.predict(X_test)
rnn_pred = rnn_model.predict(X_test)

# Convert predictions from one-hot encoding to labels
lstm_pred_labels = pos_encoder.inverse_transform(lstm_pred.argmax(axis=1))
bilstm_pred_labels = pos_encoder.inverse_transform(bilstm_pred.argmax(axis=1))
rnn_pred_labels = pos_encoder.inverse_transform(rnn_pred.argmax(axis=1))

# Evaluate models
print("LSTM Model Evaluation:")
print(classification_report(y_test['POS'], lstm_pred_labels))

print("BiLSTM Model Evaluation:")
print(classification_report(y_test['POS'], bilstm_pred_labels))

print("RNN Model Evaluation:")
print(classification_report(y_test['POS'], rnn_pred_labels))
